In [ ]:
from core.strategies import BacktestManager
from core.signals import OECDUSLEIHP
signal = OECDUSLEIHP.from_fred_data()

bt = BacktestManager.from_universe(start="2010-1-1", commission=10, shares_frac=0)
bt.MinCorr()
bt.EqualWeight()
bt.RegimeRotation(signal=signal)
bt.analytics
bt.values.plot(figsize=(12, 10))